In [ ]:
import pandas as pd
import numpy as np
import os 
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()
FOLDER_PATH = os.getenv("FOLDER_PATH")

if not FOLDER_PATH:
    raise ValueError("FOLDER_PATH not set in .env file!")

In [2]:
df = pd.read_csv(os.path.join(FOLDER_PATH, "data", "products_and_ingredients.csv")).iloc[:, 1:]
print(df.shape)
print(df.columns)
df.head()

(3549, 6)
Index(['prod_name', 'brand_name', 'ingreds', 'ratingscore', 'skin_type',
       'concerns'],
      dtype='object')


,prod_name,brand_name,ingreds,ratingscore,skin_type,concerns
0,Lip Butter Balm for Hydration & Shine,Summer Fridays,"Phytosteryl/Behenyl Dimer Dilinoleate, Diisost...",4.4,NaN,Dryness and Dullness
1,Watermelon Glow PHA + BHA Pore-Tight Toner,Glow Recipe,"Opuntia Ficus-Indica Stem Extract, Citrullus L...",4.3,Normal Dry Combination Oily,"Pores, Dryness, and Dullness"
2,Power Mist Hydrating Hand Sanitizer,Touchland,"Alcohol, Deionized/Demineralized Water, Aloe B...",4.2,NaN,NaN
3,Hyaluronic Acid 2% + B5 Hydrating Serum,The Ordinary,"Aqua/Water/Eau, Hydrolyzed Sodium Hyaluronate,...",4.5,Normal Dry Combination Oily,Dryness
4,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,"Diisostearyl Malate, Hydrogenated Polyisobuten...",4.3,Normal Dry Combination Oily,"Fine Lines/Wrinkles, Dryness, and Dullness"


In [5]:
def column_summary(df):
    
    summary_df = pd.DataFrame({
        'col_name' : df.columns,
        'col_dtype' : df.dtypes.values,
        'num_of_nulls' : df.isnull().sum().values,
        'null%' : round((df.isnull().sum()/df.shape[0])*100, 2).values,
        'num_of_non_nulls' : df.count().values,
        'num_of_distinct_values' : df.nunique().values
    })
    
    ## extracting distinct_values_counts for summary_df
    distinct_values_counts = []
    for i in range(df.shape[1]):
        if summary_df['num_of_distinct_values'][i]<=10:
            distinct_values_counts.append(dict(zip(df[df.columns[i]].value_counts().index.to_list(),df[df.columns[i]].value_counts().values)))
        else:
            distinct_values_counts.append(dict(zip(df[df.columns[i]].value_counts().index.to_list()[:10],df[df.columns[i]].value_counts().values[:10])))
            
    summary_df['distinct_values_counts'] = distinct_values_counts
    
    return summary_df

In [43]:
column_summary(df)

,col_name,col_dtype,num_of_nulls,null%,num_of_non_nulls,num_of_distinct_values,distinct_values_counts
0,Unnamed: 0,int64,0,0.00,3549,3549,"{3548: 1, 0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, ..."
1,prod_name,object,0,0.00,3549,3368,"{'The Lifting and Firming Mask': 2, 'Cucumber ..."
2,brand_name,object,0,0.00,3549,279,"{'CLINIQUE': 166, 'SEPHORA COLLECTION': 108, '..."
3,ingreds,object,0,0.00,3549,3188,"{'No Info': 23, 'Visit the Dior boutique': 18,..."
4,ratingscore,float64,0,0.00,3549,35,"{4.4: 422, 4.3: 392, 4.5: 366, 4.2: 328, 4.6: ..."
5,skin_type,object,1102,31.05,2447,31,"{'Normal Dry Combination Oily': 1256, 'Normal ..."
6,concerns,object,2121,59.76,1428,388,"{'Fine Lines and Wrinkles, Dryness, and Loss o..."


In [9]:
df.duplicated().sum()

np.int64(0)

### Data Cleaning :

In [ ]:
df['skin_profile'] = (df['skin_type'].astype(str).replace('nan', '') +' concerns: '+ df['concerns'].astype(str).replace('nan', '')).replace(' concerns: ', np.nan)
df.drop(['skin_type', 'concerns'], axis=1, inplace=True)

### Separating Products and Ingredients Data :

In [ ]:
df.head()

,prod_name,brand_name,ingreds,ratingscore,skin_profile
0,Lip Butter Balm for Hydration & Shine,Summer Fridays,"Phytosteryl/Behenyl Dimer Dilinoleate, Diisost...",4.4,concerns: Dryness and Dullness
1,Watermelon Glow PHA + BHA Pore-Tight Toner,Glow Recipe,"Opuntia Ficus-Indica Stem Extract, Citrullus L...",4.3,"Normal Dry Combination Oily concerns: Pores, D..."
2,Power Mist Hydrating Hand Sanitizer,Touchland,"Alcohol, Deionized/Demineralized Water, Aloe B...",4.2,NaN
3,Hyaluronic Acid 2% + B5 Hydrating Serum,The Ordinary,"Aqua/Water/Eau, Hydrolyzed Sodium Hyaluronate,...",4.5,Normal Dry Combination Oily concerns: Dryness
4,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,"Diisostearyl Malate, Hydrogenated Polyisobuten...",4.3,Normal Dry Combination Oily concerns: Fine Lin...


In [6]:
import re
def clean_text(text):
    if not isinstance(text, str):
        return ""
    return re.sub(r'[^a-z0-9\s]', '', text.lower().strip())

In [10]:
df['prod_descrp'] = df['brand_name'].apply(clean_text) +' '+ df['prod_name'].apply(clean_text)
df.columns

Index(['prod_name', 'brand_name', 'ingreds', 'ratingscore', 'skin_profile',
       'prod_descrp'],
      dtype='object')

In [23]:
df['list_of_ingreds'] = df['ingreds'].str.split(", ")

In [30]:
ingreds_data = df.explode('list_of_ingreds').rename(columns={'list_of_ingreds': 'ingred_name'})[['ingred_name', 'ratingscore', 'skin_profile']].sample(113224).reset_index(drop=True)

In [33]:
prods_data = df[['brand_name', 'prod_name', 'prod_descrp', 'list_of_ingreds']]

In [ ]:
prods_data['list_of_ingreds'] = prods_data['list_of_ingreds'].apply(lambda x: x.replace('[', '').replace(']', ''))
prods_data['list_of_ingreds'] = prods_data['list_of_ingreds'].apply(lambda x: x.replace("'", ""))

In [ ]:
ingreds_data.to_csv(os.path.join(FOLDER_PATH, 'data', 'ingredients_data.csv'), index=False)
prods_data.to_csv(os.path.join(FOLDER_PATH, 'data', 'products_data.csv'), index=False)